In [1]:
import pandas as pd
from obspy import UTCDateTime
from obspy.clients.fdsn import Client as FDSNclient
from src.filtering import filtering_data
from src.fourier import fourier_transform
from src.core import main_processing
from obspy.clients.seedlink.easyseedlink import create_client
from obspy import Trace, UTCDateTime, Stream
from obspy.clients.fdsn import Client as FDSNclient
from obspy.core.inventory import Inventory
import matplotlib.pyplot as plt
import numpy as np
import logging

def process_index(element, ref_time):
    event_stat = {}
    try:
    
        url = 'rtserve.iris.washington.edu:18000'
        source = {
            "LX": {"MORF": ["BHE", "BHN", "BHZ"]}
        }

        loc = '--'
        
        client = FDSNclient("IRIS")
        gap = 120
        
        ref_time = UTCDateTime(ref_time)

        starttime = ref_time - gap  # 2 minutes before
        endtime = ref_time + gap  # 2 minutes after

        st = None
        
        # Attempt to print available streams
        for server, values in source.items():
            for channel, ch in values.items():
                for i in ch:
                    trace = client.get_waveforms(server, channel, loc, i, starttime, endtime, attach_response=True)
                    
                    if st is None:
                        st = trace
                    else:
                        st += trace  # Use += to concatenate traces

        if st is None:
            logging.error("No data retrieved for the given time window.")
            return {}
        
        st2 = st.copy()

        # Process components
        st_z = st.select(component="Z")
        st_n = st.select(component="N")
        st_e = st.select(component="E")

        # Process Z component
        if len(st_z) > 0:
            filtered_z, time_vector_z = filtering_data(st2.select(component="Z")[0])
            dom_freq_z, amp_z = fourier_transform(filtered_z, st_z[0])
            event_stat["Domin_freq_z"] = dom_freq_z
        else:
            print("No Z component traces available.")

        # Process N component
        if len(st_n) > 0:
            filtered_n, time_vector_n = filtering_data(st2.select(component="N")[0])
            dom_freq_n, amp_n = fourier_transform(filtered_n, st_n[0])
            event_stat["Domin_freq_n"] = dom_freq_n
        else:
            print("No N component traces available.")

        # Process E component
        if len(st_e) > 0:
            filtered_e, time_vector_e = filtering_data(st2.select(component="E")[0])
            dom_freq_e, amp_e = fourier_transform(filtered_e, st_e[0])
            event_stat["Domin_freq_e"] = dom_freq_e
        else:
            print("No E component traces available.")

        details = main_processing(filtered_z, gap, dom_freq_z)
        display(details)

        for key, dicts in details.items():
            if key == 1:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        event_stat["P-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        event_stat["P-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        event_stat["P-peak2peak"] = values
                    elif stat == "r":
                        event_stat["P-r"] = values
                    elif stat == "moment_history":
                        event_stat["P-moment_history"] = values
                    elif stat == "tau_c":
                        event_stat["P-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == 2:
                for stat, values in dicts.items():
                    if stat == "Vel Amp (m/s)":
                        event_stat["S-Vel amp (m/s)"] = values
                    elif stat == "Disp Amp (m)":
                        event_stat["S-Disp amp (m)"] = values
                    elif stat == "peak2peak":
                        event_stat["S-peak2peak"] = values
                    elif stat == "r":
                        event_stat["S-r"] = values
                    elif stat == "moment_history":
                        event_stat["S-moment_history"] = values
                    elif stat == "tau_c":
                        event_stat["S-tau_c"] = values    
                    else:
                        print(f"{key} error")
            elif key == "RMS":
                event_stat["RMS"] = dicts    
            elif key == "Energy":
                event_stat["Energy"] = dicts 
            elif key == "peak_freq":
                event_stat["peak_freq"] = dicts 
            elif key == "wavelength":
                event_stat["wavelength"] = dicts 
            elif key == "peak_disp":
                event_stat["peak_disp"] = dicts 
            elif key == "M0":
                event_stat["M0"] = dicts 
            else:
                event_stat["other infos"] = values
                
        print(f"...Finished processing {element}")    
        return event_stat
    except Exception as e:
        print(f"Error fetching index: {element}, Error: {e}")

In [3]:
import numpy as np
import joblib
from sklearn.preprocessing import MinMaxScaler

# Load the model from the .pkl file
model = joblib.load('models/model_FINAL.pkl')
best_model = model.best_estimator_

#events: 

event = {"1": "2024-11-06 16:03:17" # M3.4 "close" sensed-n = 0
         ,"2": "2024-09-04 00:03:00" # M1.9 "close" int = II sensed-n = 1
         ,"3": "2024-06-29 04:45:00" # M3.4         III/IV sensed-n = 1
         ,"5": "2024-08-26 05:11:00" # M5.3 "close" int IV/V sensed-n = 1
         ,"4": "2024-08-11 07:18:00" # M2.6 "close" III sensed-n = 1
         ,"6": "2024-08-11 05:48:00" # M3.4 "close" III sensed-n = 1
        }

# pre-processing

event_data = pd.DataFrame()
for key, values in event.items():
    event_stat = process_index(key, values)
    event_data["datetime"] = values
    event_data = pd.concat([event_data, pd.Series(event_stat, name=key).to_frame().T], ignore_index=True, axis=0)
    # Reset index to have a clean DataFrame
    event_data.reset_index(drop=True, inplace=True)
    
event_data["Domin_freq_n"] = abs(event_data["Domin_freq_n"])
event_data["Domin_freq_e"] = abs(event_data["Domin_freq_e"])
event_data["Domin_freq_z"] = abs(event_data["Domin_freq_z"])
event_data["P-Vel amp (m/s)"] = abs(event_data["P-Vel amp (m/s)"])
event_data["S-Vel amp (m/s)"] = abs(event_data["S-Vel amp (m/s)"])
event_data["P-Disp amp (m)"] = abs(event_data["P-Disp amp (m)"])
event_data["S-Disp amp (m)"] = abs(event_data["S-Disp amp (m)"])
event_data["Mw"] = 4.525*np.log10(event_data["P-tau_c"]) +5.036

# select the features
features = event_data [['Domin_freq_z','P-peak2peak', 'P-Disp amp (m)', 'P-r', 'P-moment_history', 'P-tau_c','Mw']]

# normalize
normalizer = MinMaxScaler()
normalizer.fit(features)
features_norm = normalizer.transform(features)
features_norm = pd.DataFrame(features_norm, columns=features.columns)

for index, event in features_norm.iterrows():      
    y_prob = best_model.predict_proba([event])[:, 1]  # Probabilities for the positive class
    custom_threshold = 0.01
    y_pred_custom_threshold = (y_prob >= custom_threshold).astype(int)
    print(y_pred_custom_threshold)
    if y_pred_custom_threshold == 1:
        print("ALERT!")             

{1: {'peak2peak': np.float64(0.06666666666666667),
  'Vel Amp (m/s)': np.float64(6.82575779100653e-07),
  'Disp Amp (m)': np.float64(-1.663846472679064e-08),
  'r': np.float64(3.963250996061087e-06),
  'moment_history': np.float64(2.638282172642141e-08),
  'tau_c': np.float64(3156.1241591444214)},
 2: {'peak2peak': np.float64(0.0761904761904762),
  'Vel Amp (m/s)': np.float64(2.840349027226505e-05),
  'Disp Amp (m)': np.float64(-6.923633763206453e-07)},
 'RMS': np.float64(1.8021782788252969e-06),
 'Energy': np.float64(3.2478465486697096e-12),
 'peak_freq': np.float64(6.529166666666667),
 'wavelength': np.float64(918.9534141671985),
 'peak_disp': np.float64(2.0043688434520157e-08),
 'M0': np.float64(6013106530.356047)}

...Finished processing 1


{1: {'peak2peak': np.float64(0.14545454545454545),
  'Vel Amp (m/s)': np.float64(2.111295162702664e-08),
  'Disp Amp (m)': np.float64(1.001807885128357e-09),
  'r': np.float64(3.2982769707178277e-06),
  'moment_history': np.float64(-1.4501574514206789e-09),
  'tau_c': np.float64(3459.6829966070163)},
 2: {'peak2peak': np.float64(0.16),
  'Vel Amp (m/s)': np.float64(2.959681869592324e-08),
  'Disp Amp (m)': np.float64(1.4043667066586258e-09)},
 'RMS': np.float64(5.808716337551771e-09),
 'Energy': np.float64(3.374118549014086e-17),
 'peak_freq': np.float64(3.3541666666666665),
 'wavelength': np.float64(1788.8198757763976),
 'peak_disp': np.float64(-2.134588811681685e-10),
 'M0': np.float64(-64037664.35045055)}

...Finished processing 2


{1: {'peak2peak': np.float64(0.14545454545454545),
  'Vel Amp (m/s)': np.float64(1.1464400959204441e-08),
  'Disp Amp (m)': np.float64(5.270747178614645e-10),
  'r': np.float64(1.5893550793999137e-07),
  'moment_history': np.float64(7.519360069248243e-11),
  'tau_c': np.float64(15760.478586802097)},
 'RMS': np.float64(4.21307814939779e-09),
 'Energy': np.float64(1.7750027492933104e-17),
 'peak_freq': np.float64(3.461778796084149),
 'wavelength': np.float64(1733.2129963898917),
 'peak_disp': np.float64(6.748270494204698e-11),
 'M0': np.float64(20244811.482614093)}

...

{1: {'peak2peak': np.float64(0.14545454545454545),
  'Vel Amp (m/s)': np.float64(7.769828827423235e-09),
  'Disp Amp (m)': np.float64(3.6784068141288505e-10),
  'r': np.float64(7.349436038318287e-08),
  'moment_history': np.float64(-2.5620289835891787e-10),
  'tau_c': np.float64(23176.767336136436)},
 2: {'peak2peak': np.float64(0.14545454545454545),
  'Vel Amp (m/s)': np.float64(2.556245664582901e-08),
  'Disp Amp (m)': np.float64(1.2101825767386216e-09)},
 'RMS': np.float64(4.043339698497421e-09),
 'Energy': np.float64(1.6348595917445215e-17),
 'peak_freq': np.float64(3.361799625078109),
 'wavelength': np.float64(1784.7583643122675),
 'peak_disp': np.float64(7.555420785325056e-11),
 'M0': np.float64(22666262.355975166)}

...Finished processing 6
[1]
ALERT!
[1]
ALERT!
[1]
ALERT!
[0]


c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.w

[1]
ALERT!
[1]
ALERT!


c:\Users\Utilizador\Desktop\IRONHACK\.venv\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but BaggingClassifier was fitted with feature names
  warnings.warn(
